# <center>Matching as a LCP</center>
### <center>Alfred Galichon (NYU & Sciences Po)</center>
## <center>'math+econ+code' masterclass</center>

## The Lemke-Howson algorithm

### References

XXX

In [1]:
#!pip install gurobipy
import numpy as np
import scipy.sparse as spr
import gurobipy as grb
import sympy
import math
import lemkelcp as lcp
from sympy.solvers import solve
from sympy import *

def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(Number)})

# Setting up and solving the LCP

We shall solve the problem as a LCP.

In [2]:
class LCP: # z >=0, w = M z + q >= 0, z.w=0
    def __init__(self,M_k_k,q_k):
        self.M_k_k = M_k_k
        self.nbk,_ = M_k_k.shape
        self.q_k = q_k
    
    def qp_solve(self,silent = True):
        qp = grb.Model()
        if silent:
            qp.Params.OutputFlag = 0
        qp.Params.NonConvex = 2
        zqp_k = qp.addMVar(shape = self.nbk)
        wqp_k = qp.addMVar(shape = self.nbk)
        qp.addConstr(self.M_k_k @ zqp_k - wqp_k == - self.q_k)
        qp.setObjective(zqp_k @ wqp_k, sense = grb.GRB.MINIMIZE)
        qp.optimize()
        z_k = np.array(qp.getAttr('x'))[:self.nbk]
        return(z_k)         

In [3]:
kostrava_ex = LCP(np.array([[ 0,  3,  1],[-3,  2, -2],[-1, -2,  2]]) , np.array([1,-2,2]))
zsol = kostrava_ex.qp_solve()
print(zsol)
wsol = kostrava_ex.M_k_k @ zsol + kostrava_ex.q_k
print(wsol)
print(zsol @ wsol)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-23
[0. 1. 0.]
[4. 0. 0.]
0.0


In [4]:
murty_ex_2_10 = LCP(np.array([[ 1,0, 0],[2, 1,0],[2,2,1]]) , np.array([-8,-12,-14]))

## Solving LCP using lemkelcp

In [5]:
#def NashEquilibrium_lcplemke_solve(self, silent = True):
#    sol_z = self.lcp_pb.solve_lemkelcp()
#    x_i = sol_z[:self.nbi]
#    y_j = sol_z[self.nbi:]
#    val1 = 1 / y_j.sum()
#    val2 = 1 /  x_i.sum()
#    p_i = x_i * val2
#    q_j = y_j * val1
#    sol_dict = {'val1':val1, 'val2':val2, 'p_i':p_i,'q_j':q_j}
#    return(sol_dict)

#NashEquilibrium.lcplemke_solve = NashEquilibrium_lcplemke_solve

# Lemke algorithm

# Tableaux

In [6]:
class Tableau():
    def __init__(self, names_basic,names_nonbasic, M_i_j,c_i,obj_j = None): #z_B = c - M @ z_N
        self.names_initial_basic = names_basic
        self.names_initial_nonbasic = names_nonbasic
        self.nonbasic = list(symbols(names_nonbasic))
        if obj_j is None:
            self.base = dict()
        else:
            self.base = { Symbol('obj') : obj_j @ self.nonbasic }
        basic = list(symbols(names_basic))
        self.base.update( { basic[i]: c_i[i]  - (M_i_j @ self.nonbasic)[i]  for i in range(len(c_i))} ) 
        
        
    def variables(self):
        return( list(self.base.keys())[1:] +self.nonbasic) 

In [7]:
def Tableau_determine_departing(self,entering_var):
    runmin = float('inf')
    departing_var = None 
    for var in self.base.keys() - {Symbol('obj')}:
        the_expr_list = solve(self.base[var] - var,entering_var)
        if the_expr_list: # if one can invert the previous expression
            the_expr = the_expr_list[0] # express entering variable as a function of the other ones:
            val_entering_var = the_expr.subs([ (variable,0) for variable in [var]+self.nonbasic])
            if (val_entering_var > 0) & (val_entering_var < runmin) :
                runmin,departing_var = val_entering_var, var
    return departing_var # if no variable is found, None retunrned

Tableau.determine_departing = Tableau_determine_departing

In [8]:
def Tableau_determine_departing_complementary(self,entering_var):
    runmax = - float('inf')
    departing_var = None 
    for var in self.base.keys() - {Symbol('obj')}:
        the_expr_list = solve(self.base[var] - var,entering_var)
        if the_expr_list: # if one can invert the previous expression
            the_expr = the_expr_list[0] # express entering variable as a function of the other ones:
            val_entering_var = the_expr.subs([ (variable,0) for variable in [var]+self.nonbasic])
            if (val_entering_var > 0) & (val_entering_var > runmax) :
                runmax,departing_var = val_entering_var, var
    return departing_var # if no variable is found, None retunrned

Tableau.determine_departing_complementary = Tableau_determine_departing_complementary

In [9]:
def Tableau_pivot(self,entering_var,departing_var, verbose = 0):
    expr_entering = solve(self.base[departing_var] - departing_var,entering_var)[0]
    for var in self.base:
        self.base[var] = self.base[var].subs([(entering_var, expr_entering)])  
    self.base[entering_var] = expr_entering
    del self.base[departing_var]
    self.nonbasic.remove(entering_var)
    self.nonbasic.append(departing_var)
    if verbose >0:
        print('Entering = ' + str( entering_var)+'; departing = '+ str( departing_var))
    if verbose >1:
        print(str( entering_var)+' = '+str(round_expr(expr_entering,2)))
    return expr_entering

Tableau.pivot = Tableau_pivot

In [10]:
def Tableau_evaluate(self,thevar):
    if thevar in set(self.nonbasic):
        return 0.0
    else:
        return np.float(self.base[thevar].evalf(subs = {variable:0.0 for variable in self.nonbasic} ))

Tableau.evaluate=Tableau_evaluate

def Tableau_print_solution(self,title=None):
    if not (title is None):
        print(title)
    for var in self.base:
        print(str(var)+'='+str(self.base[var].subs([ (variable,0) for variable in self.nonbasic])))

Tableau.print_solution = Tableau_print_solution

def Tableau_output_solution(self):
    xB = np.zeros(len(self.names_initial_basic))
    xN = np.zeros(len(self.names_initial_nonbasic))
    for i,name in enumerate(self.names_initial_basic):
        xB[i]=self.evaluate(Symbol(name))
    for i,name in enumerate(self.names_initial_nonbasic):
        xN[i]=self.evaluate(Symbol(name))
    return xB,xN
    
Tableau.output_solution = Tableau_output_solution

In [11]:
def LCP_lemke_howson_solve(self,verbose=2):
    zks = ['z_' + str(k+1) for k in range(self.nbk)]+['z_0']
    wks = ['w_' + str(k+1) for k in range(self.nbk)]
    tab = Tableau(wks, zks, - np.block([self.M_k_k,np.ones((self.nbk,1))]), self.q_k)
    keys = zks[:-1]+wks
    labels = wks+zks[:-1]
    complements = {Symbol(keys[t]): Symbol(labels[t]) for t in range(len(keys))}
    entering_var = Symbol('z_0')
    departing_var=tab.determine_departing_complementary(entering_var)
    while True:
        tab.pivot(entering_var,departing_var,verbose=verbose)
        if departing_var == Symbol('z_0'):
            break
        else:
            entering_var = complements[departing_var]
        departing_var = tab.determine_departing(entering_var)
    _,z_k = tab.output_solution()
    return z_k[:-1]

LCP.lemke_howson_solve = LCP_lemke_howson_solve

In [12]:
murty_ex_2_10.lemke_howson_solve()

Entering = z_0; departing = w_3
z_0 = w_3 - 2.0*z_1 - 2.0*z_2 - z_3 + 14.0
Entering = z_3; departing = w_2
z_3 = -w_2 + w_3 - z_2 + 2.0
Entering = z_2; departing = z_3
z_2 = -w_2 + w_3 - z_3 + 2.0
Entering = w_3; departing = w_1
w_3 = -w_1 + 2.0*w_2 - z_1 + z_3 + 2.0
Entering = z_1; departing = w_3
z_1 = -w_1 + 2.0*w_2 - w_3 + z_3 + 2.0
Entering = z_3; departing = z_2
z_3 = -w_2 + w_3 - z_2 + 2.0
Entering = w_2; departing = z_3
w_2 = w_3 - z_2 - z_3 + 2.0
Entering = w_3; departing = z_0
w_3 = 2.0*w_1 - z_0 + 2.0*z_2 + z_3 + 2.0


array([8., 0., 0.])

In [13]:
murty_ex_2_9 = LCP(np.array([[ 1,0, 3],[-1, 2,5],[2,1,2]]) , np.array([-3,-2,-1]))

In [14]:
murty_ex_2_9.lemke_howson_solve()

Entering = z_0; departing = w_1
z_0 = w_1 - z_1 - 3.0*z_3 + 3.0
Entering = z_1; departing = w_2
z_1 = 0.5*w_1 - 0.5*w_2 + z_2 + z_3 + 0.5
Entering = z_2; departing = z_0
z_2 = 0.5*w_1 + 0.5*w_2 - z_0 - 4.0*z_3 + 2.5


array([3. , 2.5, 0. ])

In [15]:
(murty_ex_2_9.M_k_k @  np.array([3. , 2.5, 0. ]) +murty_ex_2_9.q_k)

array([0. , 0. , 7.5])

In [16]:
murty_ex_2_9.qp_solve()

array([3.0000000e+00, 2.5000000e+00, 2.2014867e-10])

# Matching model

In [17]:
class Matching_market():
    def __init__(self, n_x=None,m_y=None):
        if n_x is None:
            self.n_x = np.ones(self.nbx)
        else:
            self.n_x = n_x
        if m_y is None:
            self.m_y = np.ones(self.nby)
        else:
            self.m_y = m_y
        self.nba = self.nbx * self.nby + self.nbx + self.nby  
        self.MOM_z_xy = spr.bmat([[spr.kron(spr.identity(self.nbx), np.ones((1,self.nby)))],
                                 [spr.kron(np.ones((1,self.nbx)),spr.identity(self.nby))]]) 
        
        
    def lcp_solve(self,silent=True,precision = 16):
        qp = grb.Model()
        if silent:
            qp.Params.OutputFlag = 0
        qp.Params.NonConvex = 2
        self.qp_setup(qp)
        qp.optimize()
        sol = qp.getAttr('x')
        ustar_x = np.array(sol[:self.nbx])
        vstar_y = np.array(sol[self.nbx:(self.nbx+self.nby)])
        mustar_x_y =np.array(sol[(self.nbx+self.nby):(self.nbx+self.nby+self.nbx*self.nby)]).reshape((self.nbx,self.nby)) 
        return {'scalprod':qp.getAttr('objval'),
                'u_x': ustar_x.round(precision),
                'v_y': vstar_y.round(precision),
                'mu_x_y':mustar_x_y.round(precision)}

### TU markets

In [18]:
class TU_market(Matching_market):
    def __init__(self,Φ_x_y,n_x=None,m_y = None):
            self.nbx,self.nby = Φ_x_y.shape
            self.Φ_x_y = Φ_x_y
            Matching_market.__init__(self,n_x,m_y)
        
    def qp_setup(self,qp):
        nbx,nby = self.nbx,self.nby

        p_z = qp.addMVar(shape = nbx+nby)
        mu_xy = qp.addMVar(shape = nbx*nby)

        dualp_z = qp.addMVar(shape = nbx+nby)
        dualmu_xy = qp.addMVar(shape = nbx*nby)

        qp.addConstr(dualp_z == np.concatenate([self.n_x,self.m_y]) - self.MOM_z_xy @ mu_xy )
        qp.addConstr(dualmu_xy == self.MOM_z_xy.T @ p_z -self.Φ_x_y.flatten() )

        qp.setObjective(p_z @ dualp_z + mu_xy @ dualmu_xy   , sense = grb.GRB.MINIMIZE)
        return(qp)


In [19]:
TU_ex=TU_market(np.array([[.2,.6],[.3,-.1],[.7,.5]])+np.array([[.7,.3],[.3,.6],[0.2,-.2]]))
TU_ex.lcp_solve(precision=5)

{'scalprod': 2.343516718615645e-08,
 'u_x': array([0.25053, 0.     , 0.09625]),
 'v_y': array([0.80375, 0.64947]),
 'mu_x_y': array([[0., 1.],
        [0., 0.],
        [1., 0.]])}

### LTU markets

In [20]:
class LTU_market(Matching_market):
    def __init__(self,ζ_x_y,Φ_x_y,n_x=None,m_y = None):
            self.nbx,self.nby = Φ_x_y.shape
            self.Φ_x_y = Φ_x_y
            self.ζ_x_y = ζ_x_y
            Matching_market.__init__(self,n_x,m_y)

    def qp_setup(self,qp):
        nbx,nby = self.nbx,self.nby

        p_z = qp.addMVar(shape = nbx+nby)
        mu_xy = qp.addMVar(shape = nbx*nby)

        dualp_z = qp.addMVar(shape = nbx+nby)
        dualmu_xy = qp.addMVar(shape = nbx*nby)

        zetaMXT = spr.diags(self.ζ_x_y.flatten()) @ spr.kron(spr.identity(self.nbx), np.ones((self.nby,1)))
        oneminuszetaMYT = spr.diags(1 - self.ζ_x_y.flatten()) @ spr.kron(np.ones((self.nbx,1)),spr.identity(self.nby))
        
        qp.addConstr(dualp_z == np.concatenate([self.n_x,self.m_y]) - self.MOM_z_xy @ mu_xy )
        qp.addConstr(dualmu_xy == spr.hstack([zetaMXT,oneminuszetaMYT]) @ p_z -self.Φ_x_y.flatten()/2 )

        qp.setObjective(p_z @ dualp_z + mu_xy @ dualmu_xy , sense = grb.GRB.MINIMIZE)
        return(qp)


In [21]:
LTU_ex=LTU_market(np.array([[.5,.5],[.5,.5],[.5,.5]]),np.array([[0.9, 0.9],[0.6, 0.5],[0.9, 0.3]])/2)
LTU_ex.lcp_solve(precision=5)

{'scalprod': 2.425545405086363e-08,
 'u_x': array([0.10396, 0.     , 0.02131]),
 'v_y': array([0.42869, 0.34604]),
 'mu_x_y': array([[0., 1.],
        [0., 0.],
        [1., 0.]])}

### NTU markets

In [22]:
class NTU_market(Matching_market):
    def __init__(self,α_x_y,γ_x_y,n_x=None,m_y = None):
        self.nbx,self.nby = α_x_y.shape
        self.α_x_y = α_x_y
        self.γ_x_y = γ_x_y
        Matching_market.__init__(self,n_x,m_y)

    
    def qp_setup(self,qp):
        nbk = 2
        nbx,nby = self.nbx,self.nby
        
        p_z = qp.addMVar(shape = nbx+nby)
        mu_xy = qp.addMVar(shape = nbx*nby)
        pi_kxy = qp.addMVar(shape = (nbk*nbx*nby))
        D_xy = qp.addMVar(shape = nbx*nby, lb = -grb.GRB.INFINITY)

        dualp_z = qp.addMVar(shape = nbx+nby)
        dualmu_xy = qp.addMVar(shape = nbx*nby)
        dualpi_kxy = qp.addMVar(shape = (nbk*nbx*nby))

        theDks = [spr.hstack([spr.kron(spr.identity(nbx), np.ones((nby,1))),spr.csr_matrix((nbx*nby,nby))]),
                spr.hstack([spr.csr_matrix((nbx*nby,nbx)), spr.kron(np.ones((nbx,1)),spr.identity(nby))])]
        the_cks = [self.α_x_y.flatten(),self.γ_x_y.flatten()]

        qp.addConstr(dualp_z == np.concatenate([self.n_x,self.m_y]) - self.MOM_z_xy @ mu_xy )
        qp.addConstr(dualmu_xy == D_xy)
        for k in range(nbk):
            qp.addConstr(dualpi_kxy[(k*nbx*nby):((k+1)*nbx*nby)] == D_xy - theDks[k] @ p_z + the_cks[k]  ) 
        qp.addConstr(spr.kron(np.ones((1,nbk)), spr.identity(nbx*nby)) @ pi_kxy == np.ones((1,nbx*nby)))

        qp.setObjective(p_z @ dualp_z + mu_xy @ dualmu_xy + dualpi_kxy @ pi_kxy  , sense = grb.GRB.MINIMIZE)
        return(qp)


In [23]:
NTU_ex=NTU_market(np.array([[.2,.6],[.3,-.1],[.7,.5]]),
                  np.array([[.7,.3],[.3,.6],[0.2,-.2]]))
NTU_ex.lcp_solve(precision=5)


{'scalprod': 5.398852163033417e-08,
 'u_x': array([0.26858, 0.05265, 0.     ]),
 'v_y': array([0.3, 0.3]),
 'mu_x_y': array([[0., 1.],
        [1., 0.],
        [0., 0.]])}

### ETU markets

In [24]:
class ETU_market(Matching_market):
    def __init__(self,α_x_y,γ_x_y,τ,n_x=None,m_y = None):
        self.nbx,self.nby = α_x_y.shape
        self.α_x_y = α_x_y
        self.γ_x_y = γ_x_y
        self.τ = τ
        Matching_market.__init__(self,n_x,m_y)
    
    def qp_setup(self,qp):
        nbx,nby = self.nbx,self.nby
        ζ_x_y = np.exp(- self.α_x_y / self.τ) / (np.exp(- self.α_x_y / self.τ) + np.exp(- self.γ_x_y / self.τ))
        Φ_x_y = 4 / (np.exp(- self.α_x_y / self.τ) + np.exp(- self.γ_x_y / self.τ)) - 2
        p_z = qp.addMVar(shape = nbx+nby)
        mu_xy = qp.addMVar(shape = nbx*nby)

        dualp_z = qp.addMVar(shape = nbx+nby)
        dualmu_xy = qp.addMVar(shape = nbx*nby)

        zetaMXT = spr.diags(ζ_x_y.flatten()) @ spr.kron(spr.identity(self.nbx), np.ones((self.nby,1)))
        oneminuszetaMYT = spr.diags(1 - ζ_x_y.flatten()) @ spr.kron(np.ones((self.nbx,1)),spr.identity(self.nby))
        
        qp.addConstr(dualp_z == np.concatenate([self.n_x,self.m_y]) - self.MOM_z_xy @ mu_xy )
        qp.addConstr(dualmu_xy == spr.hstack([zetaMXT,oneminuszetaMYT]) @ p_z - Φ_x_y.flatten()/2 )

        qp.setObjective(p_z @ dualp_z + mu_xy @ dualmu_xy , sense = grb.GRB.MINIMIZE)
        return(qp)

        
    def lcp_solve(self,silent=True,precision = 16):
        interim_sol = Matching_market.lcp_solve(self,silent,16)
        return {'scalprod':interim_sol['scalprod'],
                'u_x': (self.τ*np.log(interim_sol['u_x'])).round(precision),
                'v_y': (self.τ*np.log(interim_sol['v_y'])).round(precision),
                'mu_x_y':(interim_sol['mu_x_y']).round(precision)}

In [25]:
ETU_ex=ETU_market(np.array([[2,6],[3,1],[7,5]]),
                  np.array([[7,3],[3,6],[2,2]]),1)
ETU_ex.lcp_solve(precision=5)


{'scalprod': 1.9848958712543625e-08,
 'u_x': array([  2.60582,   1.4281 , -20.22011]),
 'v_y': array([3.52637, 3.64928]),
 'mu_x_y': array([[0.49988, 0.50012],
        [0.50012, 0.49988],
        [0.     , 0.     ]])}